In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
!git clone https://github.com/theobui1711/t2f.git

Cloning into 't2f'...
remote: Enumerating objects: 99, done.
remote: Counting objects: 100% (99/99), done.
remote: Compressing objects: 100% (71/71), done.
remote: Total 99 (delta 61), reused 66 (delta 28), pack-reused 0
Receiving objects: 100% (99/99), 227.71 KiB | 2.96 MiB/s, done.
Resolving deltas: 100% (61/61), done.


In [ ]:
!pip install --upgrade --no-cache-dir gdown

In [ ]:
!gdown 1-6VHXsqrKLcxZVzwqJ7_jJM-eruRQKsr
!gdown 1CV36iomcF85sV8ilifeVdzXdl0ydh2I4

Downloading...
From (original): https://drive.google.com/uc?id=1-6VHXsqrKLcxZVzwqJ7_jJM-eruRQKsr
From (redirected): https://drive.google.com/uc?id=1-6VHXsqrKLcxZVzwqJ7_jJM-eruRQKsr&confirm=t&uuid=7bd5c928-6b60-4792-b2db-ab6b59485e2c
To: /content/data.zip
100% 519M/519M [00:03<00:00, 130MB/s]
Downloading...
From (original): https://drive.google.com/uc?id=1CV36iomcF85sV8ilifeVdzXdl0ydh2I4
From (redirected): https://drive.google.com/uc?id=1CV36iomcF85sV8ilifeVdzXdl0ydh2I4&confirm=t&uuid=21e50e60-b91c-4a4a-9f14-2da4aeaa67ab
To: /content/t5_warmup_ep2_wob_output.zip
100% 2.47G/2.47G [00:36<00:00, 68.2MB/s]


In [ ]:
!unzip -q /content/data.zip -d /content/t2f/
!unzip -q /content/t5_warmup_ep2_wob_output.zip -d /content/t2f/

In [ ]:
!mv /content/t2f/output /content/t2f/t5_warmup_ep2_wob_output

In [ ]:
!pip -q install transformers -U accelerate -U sacremoses importlib-metadata

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.0/9.0 MB 10.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 297.6/297.6 kB 26.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 897.5/897.5 kB 50.0 MB/s eta 0:00:00


In [ ]:
%cd t2f

/content/t2f


In [ ]:
!mkdir experiments/
!mkdir experiments/full/
!echo "{}" > experiments/full/finetune_description.json

In [ ]:
# combine datasets
import json

def combine_json_files(file1_path, file2_path, output_path):
    with open(file1_path, 'r') as file1:
        data1 = json.load(file1)

    with open(file2_path, 'r') as file2:
        data2 = json.load(file2)

    combined_data = data1 + data2

    with open(output_path, 'w') as output_file:
        json.dump(combined_data, output_file)

# Replace these paths with the paths to your JSON files
file1_path = 'data/floorplan/floorplan_finetune.json'
file2_path = 'data/floorplan/floorplan_eval.json'
output_path = 'data/floorplan/floorplan_combined.json'

# combine_json_files(file1_path, file2_path, output_path)

In [ ]:
import torch
from torch.optim import Adam
from torch.optim.lr_scheduler import StepLR
from torch.utils.data import DataLoader
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, default_data_collator, \
    DataCollatorForLanguageModeling, EarlyStoppingCallback
from datasets import load_dataset
from arguments import DataTrainingArguments, ModelArguments, TrainingArguments
from collections import defaultdict
import cv2
import numpy as np
import random
import matplotlib.pyplot as plt
from utils import color_idx, room_idx

# Initialize the tokenizer
tokenizer = T5Tokenizer.from_pretrained('t5-base', legacy=True)

# Initialize the data arguments
data_args = DataTrainingArguments(
    datasets='floorplan',
    data_dir=None,
    train_split='finetune',
    max_seq_length=512,
    max_output_seq_length=512,
    max_seq_length_eval=512,
    max_output_seq_length_eval=512,
    exp='full',
    boundary_in_where='Encoder'
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.39M [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.21k [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [ ]:
model_name_or_checkpoint_path = '/content/t2f/t5_warmup_ep2_wob_output/checkpoint-34500'

if torch.cuda.is_available():
    device = torch.device('cuda')
else:
    device = torch.device('cpu')

# config = T5Config.from_pretrained('/content/t2f/t5_warmup_ep2_wob_output/checkpoint-34500', dropout_rate=0.1)

model = T5ForConditionalGeneration.from_pretrained(model_name_or_checkpoint_path)
model.to(device)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseActDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dro

In [ ]:
from datasets import load_dataset

floorplan_dataset = load_dataset(data_args.datasets, data_args, split=data_args.train_split,
                                  max_input_length=data_args.max_seq_length,
                                  max_output_length=data_args.max_output_seq_length,
                                  tokenizer=tokenizer, seed=1)

In [ ]:
class MyDataset(torch.utils.data.Dataset):
    def __init__(self, dataset, tokenizer, max_length=512):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.max_length = max_length

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        input_sentence = self.dataset[idx]['input_sentence']
        output_sentence = self.dataset[idx]['output_sentence']
        # num_room = torch.tensor(self.dataset[idx]['num_room'], dtype=torch.int64).to(device)
        # reg_labels = torch.tensor(self.dataset[idx]['regression_label'], dtype=torch.int64).to(device)
        input_ids = self.tokenizer(input_sentence, max_length=self.max_length, padding='max_length', truncation=True,
                                   return_tensors='pt')['input_ids'].squeeze()
        labels = self.tokenizer(output_sentence, max_length=self.max_length, padding='max_length', truncation=True,
                                return_tensors='pt')['input_ids'].squeeze()
        return {
            'input_ids': input_ids,
            # 'num_room': num_room,
            # 'reg_labels': reg_labels,
            'labels': labels
        }

In [ ]:
training_dataset = MyDataset(floorplan_dataset, tokenizer, max_length=512)

In [ ]:
from torch.utils.data import random_split

val_size = int(0.1 * len(training_dataset))
train_size = len(training_dataset) - val_size

train_dataset, val_dataset = random_split(training_dataset, [train_size, val_size])

In [ ]:
training_args = TrainingArguments(
    output_dir='t5_finetune_wob_output',
    overwrite_output_dir=True,
    num_train_epochs=10,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    warmup_steps=100,
    weight_decay=0.1,
    # save_steps=1000,
    save_total_limit=1,
    logging_steps=100,
    logging_dir='t5_finetune_wob_logs',
    logging_first_step=True,
    logging_strategy='steps',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    # eval_steps=1000,
    report_to='tensorboard',
    run_name='floorplan',
    load_best_model_at_end=True,
)

In [ ]:
optimizer = Adam(model.parameters(), lr=5e-4, betas=(0.9, 0.999), eps=1e-08)

In [ ]:
trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=default_data_collator,
    train_dataset=train_dataset, ####### training_dataset
    eval_dataset=val_dataset,
    optimizers=(optimizer, None),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=4)]
)

In [ ]:
%load_ext tensorboard
%tensorboard --logdir t5_finetune_wob_logs --reload_interval 30

Launching TensorBoard...

In [ ]:
trainer.train()

In [ ]:
# Example usage
input_sentence = "Balcony 1 is about 8 foot X 2 foot.  This is located right off of the kitchen. Balcony 2 is about 10 foot X 3 foot.  This is facing north of the house. The bathroom is about 8 foot X 6 foot.  It is located between the living room and the kitchen. Common room 1 is about 10 foot X 12 foot.  The common rooms are located diagonal from each other. Common room 2 is about 10 foot X 10 foot.  The common rooms are located diagonal from each other. The kitchen is a 10 foot X 10 foot room.  It looks like it is opened up to the living room. The living room is located on the east side of the house.  It is shaped like a \"T\" and it's the biggest room.  It is approximately 14 feet X 32 feet.  Each room that you come out of you will enter the living room. The master is about 10 feet X 15 feet.  It is located far west of the house.It is a rectangle shape."
input_ids = tokenizer(input_sentence, return_tensors="pt").input_ids
input_ids = input_ids.to(device)

# Generate output
output_ids = model.generate(input_ids, max_new_tokens=512)
output_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
print("Output sentence:", output_sentence)

In [ ]:
def generate_output_sentence(idx):
    example = floorplan_dataset.get_example(idx)
    input_sentence = ' '.join(example.tokens)
    input_ids = tokenizer(input_sentence, return_tensors="pt").input_ids.to(device)
    output_ids = model.generate(input_ids, max_new_tokens=512)
    output_sentence = tokenizer.decode(output_ids[0], skip_special_tokens=True)
    return example, output_sentence, None

random_idx = random.choice(range(len(floorplan_dataset)))
example, output_sentence, predicted_index = generate_output_sentence(random_idx)
print(floorplan_dataset.input_format.format_input(example, multitask=False))
print("Output sentence:", output_sentence)

In [ ]:
gt_boxes = defaultdict(list)
for room in example.rooms:
    gt_x, gt_y, gt_h, gt_w = room.x, room.y, room.h, room.w
    gt_box = [
        [int(gt_x - gt_h / 2), int(gt_y - gt_w / 2)], [int(gt_x + gt_h / 2), int(gt_y - gt_w / 2)],
        [int(gt_x - gt_h / 2), int(gt_y + gt_w / 2)], [int(gt_x + gt_h / 2), int(gt_y + gt_w / 2)]
    ]
    gt_boxes[room.type] = gt_box


res = floorplan_dataset.output_format.run_inference(example, output_sentence, predicted_index)
predicted_rooms_by_name, predicted_rooms, raw_predicted_relations, wrong_reconstruction, format_error, label_error = res

predicted_attribute = defaultdict()
for attribute_tuple in raw_predicted_relations:
    attribute_type, value, room_tuple, room_type = attribute_tuple
    if room_type not in predicted_attribute:
        predicted_attribute[room_type] = defaultdict()
    try:
        value = int(value)
    except:
        value = 128
    predicted_attribute[room_type][attribute_type] = value

# TODO: examine the predicted_attribute patterns
correct_attributes = ['x coordinate', 'y coordinate', 'height', 'width']
wrong_room = []
for room_type in predicted_attribute:
    if set(list(predicted_attribute[room_type].keys())) != set(correct_attributes):
        print('wrong output format:')
        print(predicted_attribute[room_type])
        wrong_room.append(room_type)
for wrong_r in wrong_room:
    predicted_attribute.pop(wrong_r)

all_gt_rooms = [room.type for room in example.rooms]
predicted_boxes = defaultdict()
for room in predicted_attribute:
    predicted_boxes[room] = [
        [int(predicted_attribute[room]['x coordinate'] - predicted_attribute[room]['height'] / 2),
         int(predicted_attribute[room]['y coordinate'] - predicted_attribute[room]['width'] / 2)],
        [int(predicted_attribute[room]['x coordinate'] + predicted_attribute[room]['height'] / 2),
         int(predicted_attribute[room]['y coordinate'] - predicted_attribute[room]['width'] / 2)],
        [int(predicted_attribute[room]['x coordinate'] - predicted_attribute[room]['height'] / 2),
         int(predicted_attribute[room]['y coordinate'] + predicted_attribute[room]['width'] / 2)],
        [int(predicted_attribute[room]['x coordinate'] + predicted_attribute[room]['height'] / 2),
         int(predicted_attribute[room]['y coordinate'] + predicted_attribute[room]['width'] / 2)]
    ]

for room in gt_boxes:
    y_min = gt_boxes[room][0][1]
    x_min = gt_boxes[room][0][0]
    y_max = gt_boxes[room][3][1]
    x_max = gt_boxes[room][3][0]
    gt_boxes[room] = (y_min, x_min, y_max, x_max)

for room in predicted_boxes:
    y_min = predicted_boxes[room][0][1]
    x_min = predicted_boxes[room][0][0]
    y_max = predicted_boxes[room][3][1]
    x_max = predicted_boxes[room][3][0]
    predicted_boxes[room] = (y_min, x_min, y_max, x_max)

def render_image_(example, predicted_boxes, all_gt_rooms, gt_boxes):
    image_height = 256
    image_width = 256
    number_of_color_channels = 3
    background_color = (255, 255, 255)
    gt_image = np.full((image_height, image_width, number_of_color_channels), background_color, dtype=np.uint8)
    predicted_image = np.full((image_height, image_width, number_of_color_channels), background_color, dtype=np.uint8)
    boundary_color = [0, 0, 0]

    left_boundary = int(example.boundary_tokens[1])

    living = defaultdict()
    common = defaultdict()
    master = defaultdict()
    balcony = defaultdict()
    bathroom = defaultdict()
    kitchen = defaultdict()
    storage = defaultdict()
    dining = defaultdict()
    for room in predicted_boxes:
        if room.startswith('living'):
            living[room] = predicted_boxes[room]
        elif room.startswith('common'):
            common[room] = predicted_boxes[room]
        elif room.startswith('master'):
            master[room] = predicted_boxes[room]
        elif room.startswith('balcony'):
            balcony[room] = predicted_boxes[room]
        elif room.startswith('bathroom'):
            bathroom[room] = predicted_boxes[room]
        elif room.startswith('kitchen'):
            kitchen[room] = predicted_boxes[room]
        elif room.startswith('storage'):
            storage[room] = predicted_boxes[room]
        elif room.startswith('dining'):
            dining[room] = predicted_boxes[room]
    room_type_list = [living, common, master, balcony, bathroom, kitchen, storage, dining]
    for room_type in room_type_list:
        for room in room_type:
            left_top_pr = (room_type[room][0], room_type[room][1])
            right_bt_pr = (room_type[room][2], room_type[room][3])

            color = color_idx[room_idx[room]]
            # draw room on predicted image
            cv2.rectangle(predicted_image, left_top_pr, right_bt_pr, color, -1)

    for boundary_pixel in example.boundary:
        predicted_image[boundary_pixel[0], boundary_pixel[1]] = boundary_color

    return predicted_image

In [ ]:
pre_image = render_image_(example, gt_boxes, all_gt_rooms, gt_boxes)

In [ ]:
import matplotlib.patches as mpatches


def render_image_with_legend(example, predicted_boxes, all_gt_rooms, gt_boxes):
    # Render the image as before
    image = render_image_(example, predicted_boxes, all_gt_rooms, gt_boxes)

    # Create a figure and a set of subplots
    fig, ax = plt.subplots()

    # Display the image
    ax.imshow(image)

    # Add direction labels for the entire floor plan
    plt.text(0.5, 0.02, "South", fontsize=10, ha='center', va='bottom', color='red', transform=plt.gca().transAxes)
    plt.text(0.5, 0.98, "North", fontsize=10, ha='center', va='top', color='red', transform=plt.gca().transAxes)
    plt.text(0.02, 0.5, "West", fontsize=10, ha='left', va='center', color='red', transform=plt.gca().transAxes)
    plt.text(0.98, 0.5, "East", fontsize=10, ha='right', va='center', color='red', transform=plt.gca().transAxes)

    # Create a legend
    patches = [mpatches.Patch(color=np.array(color_idx[room_idx[room]]) / 255., label=room) for room in all_gt_rooms]
    plt.legend(handles=patches, bbox_to_anchor=(1.05, 1), loc='upper left', borderaxespad=0.)

    # Show the plot
    plt.show()

In [ ]:
render_image_with_legend(example, gt_boxes, all_gt_rooms, gt_boxes)

In [ ]:
render_image_with_legend(example, predicted_boxes, all_gt_rooms, gt_boxes)

In [ ]:
!zip -r t5_finetune_wob_output.zip t5_finetune_wob_output
!zip -r t5_finetune_wob_logs.zip t5_finetune_wob_logs

In [ ]:
%cd ..

In [ ]:
!cp /content/t2f/t5_finetune2_wob_output.zip /content/drive/MyDrive/
!cp /content/t2f/t5_finetune2_wob_logs.zip /content/drive/MyDrive/

In [ ]:
from google.colab import runtime
runtime.unassign()